In [2]:
# playing around to try to make some chord plots by county
# anaya hall
# july 13, 2020

# example code copied from https://python-graph-gallery.com/231-chord-diagram-with-bokeh/
import pandas as pd
from bokeh.charts import output_file, Chord
from bokeh.io import show
from bokeh.sampledata.les_mis import data

nodes = data['nodes']
links = data['links']

nodes_df = pd.DataFrame(nodes)
links_df = pd.DataFrame(links)

source_data = links_df.merge(nodes_df, how='left', left_on='source', right_index=True)
source_data = source_data.merge(nodes_df, how='left', left_on='target', right_index=True)
source_data = source_data[source_data["value"] > 5]
source_data.columns


# chord_from_df = Chord(source_data, source="name_x", target="name_y", value="value")
# output_file('chord-diagram-bokeh.html', mode="inline")
# show(chord_from_df)

/Users/anayahall/anaconda/lib/python3.6/site-packages/bokeh/charts/builders/chord_builder.py:120: FutureWarning: The 'get_values' method is deprecated and will be removed in a future version. Use '.values' or 'np.asarray(..)' instead.
  self.matrix = m.get_values().T


In [2]:
#what is the structure of the source data
source_data

,source,target,value,name_x,group_x,name_y,group_y
1,2,0,8,Mlle.Baptistine,1,Myriel,1
2,3,0,10,Mme.Magloire,1,Myriel,1
3,3,2,6,Mme.Magloire,1,Mlle.Baptistine,1
47,23,11,9,Fantine,3,Valjean,2
49,24,11,7,Mme.Thenardier,4,Valjean,2
50,25,24,13,Thenardier,4,Mme.Thenardier,4
52,25,11,12,Thenardier,4,Valjean,2
54,26,11,31,Cosette,5,Valjean,2
57,27,11,17,Javert,4,Valjean,2
62,28,11,8,Fauchelevent,0,Valjean,2


In [4]:
# my own data
import pickle
import os
import geopandas as gpd



os.chdir('../results')

with open('c2f_FG_50p.p', 'rb') as f:
    c2f = pickle.load(f)
    
with open('f2r_FG_50p.p', 'rb') as f:
    f2r = pickle.load(f)    

# c2f

In [36]:
# convert to dataframe
c2fdf_swisno = pd.DataFrame.from_dict(c2f)
# reset index
c2fdf_swisno = c2fdf_swisno.reset_index()
# change column name to match 
c2fdf_swisno.rename(columns={'index':'SwisNo'},inplace=True)

# # grab names of counties (from column names, want as DF)
names = c2fdf_swisno.columns[1:].tolist()
names = pd.DataFrame(names, columns = ['County'])
# names = c2fdf.iloc[:,0] # old way of doing
# reshape!
# c2fdf_melted = pd.melt(c2fdf_swisno, id_vars=['SwisNo'],var_name = 'MSW_County', value_name='value')
# c2fdf_melted

#################### MOVE THIS
f2rdf = pd.DataFrame.from_dict(f2r)

# load swis data, keep only swis id number and county
swis = gpd.read_file("../data/clean/clean_swis.shp")
swis = swis[['SwisNo', 'County']]

# merge swis to get county where each facility is located
c2fdf = swis.merge(c2fdf_swisno, on = 'SwisNo')
# group by county! 
c2fdf = c2fdf.groupby(['County'], as_index = False).sum()
# merge with names so that there are rows for all
c2fdf = c2fdf.merge(names, on = 'County', how = 'outer')
# need to sort and replace NaN with 0
c2fdf.fillna(0, inplace = True)
c2fdf.sort_values(by = ['County'], inplace = True)
c2fdf.reset_index(inplace=True, drop = True)


# rename to clarify that each row is where the swis facility is located
# c2fdf.rename(columns={'County': 'SWIS_County'}, inplace = True)

# # c2fdf['value_scaled'] = round(c2fdf['value'] * 1/1000,-1)
# # c2fdf = c2fdf[c2fdf['value']>5]
# # c2fdf.reset_index(inplace=True)

# #then subset c2fdf to prepare to turn into matrix
c2fdf = c2fdf.iloc[:, 1:]

matrix = c2fdf.to_numpy() # turn into matrix array
# rows are FACILITIES (incoming waste), columns are MUNIS (outgoing waste)
matrix

# c2fdf.columns.shape
# names = names['County'].tolist()
# type(names[0])
# names

array([[87343.,     0.,     0., ...,     0.,     0.,     0.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.],
       ...,
       [    0.,     0.,     0., ..., 22145.,     0.,     0.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.],
       [    0.,     0.,     0., ...,     0.,     0., 11664.]])

In [77]:
# convert to dataframe
c2fdf_swisno = pd.DataFrame.from_dict(c2f)
# reset index
c2fdf_swisno = c2fdf_swisno.reset_index()
# change column name to match 
c2fdf_swisno.rename(columns={'index':'SwisNo'},inplace=True)

# # grab names of counties (from column names, want as DF)
names = c2fdf_swisno.columns[1:].tolist()
names = pd.DataFrame(names, columns = ['County'])
# names = c2fdf.iloc[:,0] # old way of doing

#################### MOVE THIS
f2rdf = pd.DataFrame.from_dict(f2r)

# load swis data, keep only swis id number and county
swis = gpd.read_file("../data/clean/clean_swis.shp")
swis = swis[['SwisNo', 'County']]

# merge swis to get county where each facility is located
c2fdf = swis.merge(c2fdf_swisno, on = 'SwisNo')

# merge with names so that there are rows for all
# c2fdf = c2fdf.merge(names, on = 'County', how = 'outer')
# need to sort and replace NaN with 0
# # c2fdf.fillna(0, inplace = True)
# c2fdf.sort_values(by = ['County'], inplace = True)
# c2fdf.reset_index(inplace=True, drop = True)


# rename to clarify that each row is where the swis facility is located
c2fdf.rename(columns={'County': 'SWIS_County'}, inplace = True)

# group by county! 
c2fdf = c2fdf.groupby(['SWIS_County'], as_index = False).sum()


# # reshape!
c2fdf = pd.melt(c2fdf, id_vars=['SWIS_County'],var_name = 'MSW_County', value_name='value')
# c2fdf

c2fdf = c2fdf_melted.groupby(['MSW_County', 'SWIS_County'], as_index = False).sum()

# # c2fdf['value_scaled'] = round(c2fdf['value'] * 1/1000,-1)
c2fdf = c2fdf[c2fdf['value']!=0]
c2fdf.reset_index(inplace=True)

# #then subset c2fdf to prepare to turn into matrix
# c2fdf = c2fdf.iloc[:, 1:]

# matrix = c2fdf.to_numpy() # turn into matrix array
# rows are FACILITIES (incoming waste), columns are MUNIS (outgoing waste)
# matrix

# c2fdf.columns.shape
# names = names['County'].tolist()
# type(names[0])
# names

c2fdf.rename(columns={'MSW_County': 'from', 'SWIS_County': 'to'}, inplace=True)
c2fdf = c2fdf[['from', 'to', 'value']]
c2fdf.to_csv('chord/C2F_FG_50.csv')

# with open('chord/C2F_FG_50.p', 'wb') as f:
#     pickle.dump(c2fdf, f)

type(c2fdf['from'][0])

str

In [62]:
# # from Chord import Chord


# # Example
# matrix = [
#     [0, 5, 6, 4, 7, 4],
#     [5, 0, 5, 4, 6, 5],
#     [6, 5, 0, 4, 5, 5],
#     [4, 4, 4, 0, 5, 5],
#     [7, 6, 5, 5, 0, 4],
#     [4, 5, 5, 5, 4, 0],
# ]

# matrix2 = [[1]]

# names = 'Action' #, 'Adventure', 'Comedy', 'Drama', 'Fantasy', 'Thriller']
# # type(names[0])
# Chord(matrix2, names).to_html()

KeyError: 'Action'

In [ ]:
# matrix = c2fdf.to_numpy()

c2fdf_swis

In [ ]:
matrix

AttributeError: 'list' object has no attribute 'shape'